## Importing libraries :

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt

## Data read :

In [2]:
df = pd.read_csv(r'/Users/priyankaagarwal/Downloads/finlatics/DsResearch/Media and Technology/Media and Technology/Global YouTube Statistics.csv', encoding='utf-8', encoding_errors='replace')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/priyankaagarwal/Downloads/finlatics/DsResearch/Media and Technology/Media and Technology/Global YouTube Statistics.csv'

In [ ]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
print(df.info())

## Clearing data:

In [ ]:
# print(df['Country'].describe())
# print(df['Abbreviation'].describe())

df.drop(columns=['Abbreviation'], inplace=True)

# print(df['Youtuber'].describe())
# print(df['Title'].describe())

df.drop(columns=['Title'], inplace=True)

df['subscribers']=df['subscribers'].fillna(df['subscribers'].median())
df['category']=df['category'].fillna('other')

# print(df['Country'].describe())
# print(df['Country of origin'].describe())

df.drop(columns=['Country of origin'], inplace=True)

df_country_analysis = df.dropna(subset=['Country','Population'])

columns_to_analyze = ['Country', 'Gross tertiary education enrollment (%)', 
                     'Population', 'Unemployment rate', 'Urban_population', 
                     'Latitude', 'Longitude']

for col in columns_to_analyze:
    print(f"\n=== {col} ===")
    print(f"Unique values: {df[col].nunique()}")
    print(f"Most frequent value: {df[col].mode()[0] if len(df[col].mode()) > 0 else 'N/A'}")
    print("Top 5 most frequent:")
    print(df[col].value_counts().head())
    
# print(df_country_analysis.isnull().sum())

df = df.dropna(subset=['channel_type','channel_type_rank'])

cols = ['video_views_for_the_last_30_days','subscribers_for_last_30_days']
for c in cols:
    df[c]=df[c].fillna(df[c].mean())

changed = ['created_year','created_month','created_date']
for c in changed:
    df[c]=df[c].fillna(df[c].mode()[0])

# print(df.isnull().sum())
# print(df.info())

## QUESTIONS :

### 1. What are the top 10 YouTube channels based on the number of subscribers? 

In [ ]:
print(df[['rank', 'Youtuber','subscribers']].head(10))
plt.bar(df.head(10)['Youtuber'], df.head(10)['subscribers'])
plt.xlabel('Youtuber')
plt.ylabel('Number of subscribers')
plt.xticks(rotation =90)
plt.show()

### 2. Which category has the highest average number of subscribers? 

In [ ]:
grped_data = df.groupby(['category'])['subscribers'].mean().sort_values(ascending = False)
print(grped_data.head(1)) 


### 3. How many videos, on average, are uploaded by YouTube channels in each category? 

In [ ]:
grouped_data = df.groupby(['category'])['uploads'].mean()
print(grouped_data)
plt.plot(grouped_data.index, grouped_data.values)
plt.xlabel('Category')
plt.ylabel('Videos on average uploaded by channels in each category')
plt.xticks(rotation =90)
plt.show()

### 4. What are the top 5 countries with the highest number of YouTube channels?  

In [ ]:
grouped_data_country = df_country_analysis.groupby(['Country'])['Youtuber'].count().sort_values(ascending = False)
print(grouped_data_country.head(5))

plt.bar(grouped_data_country.head(5).index, grouped_data_country.head(5).values)
plt.xlabel('Country')
plt.ylabel('No. of YouTube channels')
plt.show()

### 5. What is the distribution of channel types across different categories?  

In [ ]:
grouped_data_categories = df.groupby(['category','channel_type'])['channel_type'].count()
print(grouped_data_categories)


### 6. Is there a correlation between the number of subscribers and total video views for YouTube channels? 

In [ ]:
correlation = df['subscribers'].corr(df['video views'])
print("correlation: ",correlation)

if correlation > 0.9:
    print("Very high positive correlation")
elif correlation > 0.7:
    print("High positive correlation")
elif correlation > 0.5:
    print("Moderate high positive correlation")
elif correlation > 0.3:
    print("Low positive correlation")
elif correlation > 0.0:
    print("negligible correlation")
elif correlation == 0:
    print("No correlation")
elif correlation > -0.3:
    print("Negligible correlation")
elif correlation > -0.5:
    print("Low negative correlation")
elif correlation > -0.7:
    print("Moderate negative correlation")
elif correlation > -0.9:
    print("High negative correlation")
else:
    print("Very high negative correlation.")

### 7. How do the monthly earnings vary throughout different categories?  

In [ ]:
earnings_stats = df.groupby('category')[['lowest_monthly_earnings', 'highest_monthly_earnings']].agg(['mean', 'median', 'std', 'min', 'max'])
print(earnings_stats)

### 8. What is the overall trend in subscribers gained in the last 30 days across all channels? 

In [ ]:
print(df['subscribers_for_last_30_days'].describe())
sns.barplot(x='Youtuber',y='subscribers_for_last_30_days',data=df)
plt.xticks(rotation=45, ha='right')
plt.xlabel('YouTube Channels')
plt.ylabel('Subscribers Gained (Last 30 Days)')
plt.title('Subscriber Growth Trend Across All Channels (Last 30 Days)')
plt.show()

### 9. Are there any outliers in terms of yearly earnings from YouTube channels?  

In [ ]:
sns.boxplot(y='lowest_yearly_earnings',data=df)
plt.title("outliers in lowest_yearly_earnings")
plt.show()

sns.boxplot(y='highest_yearly_earnings',data=df)
plt.title("outliers in highest_yearly_earnings")
plt.show()

q1_low = df['lowest_yearly_earnings'].quantile(0.25)
q3_low = df['lowest_yearly_earnings'].quantile(0.75)
iqr_low = q3_low - q1_low

q1_high = df['highest_yearly_earnings'].quantile(0.25)
q3_high = df['highest_yearly_earnings'].quantile(0.75)
iqr_high = q3_high - q1_high

outliers_low = df[(df['lowest_yearly_earnings']<q1_low-1.5*iqr_low) | (df['lowest_yearly_earnings']>q3_low+1.5*iqr_low)]
outliers_high = df[(df['highest_yearly_earnings']<q1_high-1.5*iqr_high) | (df['highest_yearly_earnings']>q3_high+1.5*iqr_high)]

print(f"Outliers in lowest yearly earnings: {len(outliers_low)}")
print(f"Outliers in highest yearly earnings: {len(outliers_high)}")


### 10. What is the distribution of channel creation dates? Is there any trend over time? 

In [ ]:
print(df['created_date'].describe())

grouped_data_year = df.groupby(['created_year','created_month'])['Youtuber'].count()
print(grouped_data_year)

yearly_counts = df.groupby('created_year')['Youtuber'].count()
yearly_counts.plot(kind = 'bar')
plt.xlabel('year')
plt.ylabel('number of channels created')
plt.title('Trend in YouTube Channel Creation Over Time')
plt.show()

### 11. Is there a relationship between gross tertiary education enrollment and the number of YouTube channels in a country?  

In [ ]:
data_country = df.groupby(['Country']).agg({'Youtuber': 'count','Gross tertiary education enrollment (%)': 'first', 'Unemployment rate': 'first', 'Country': 'first', 'Population':'first'})
relation = data_country['Youtuber'].corr(data_country['Gross tertiary education enrollment (%)'])
print(relation)

if relation > 0.9:
    print("Very high positive correlation")
elif relation > 0.7:
    print("High positive correlation")
elif relation > 0.5:
    print("Moderate high positive correlation")
elif relation > 0.3:
    print("Low positive correlation")
elif relation > 0.0:
    print("negligible correlation")
elif relation == 0:
    print("No correlation")
elif relation > -0.3:
    print("Negligible correlation")
elif relation > -0.5:
    print("Low negative correlation")
elif relation > -0.7:
    print("Moderate negative correlation")
elif relation > -0.9:
    print("High negative correlation")
else:
    print("Very high negative correlation.")

### 12. How does the unemployment rate vary among the top 10 countries with the highest number of YouTube channels?

In [ ]:
top10 = data_country.sort_values('Youtuber', ascending = False).head(10)
plt.plot(top10['Country'], top10['Unemployment rate'])
plt.xlabel('Country')
plt.ylabel('Unemployment rate')
plt.xticks(rotation=90)
plt.title('Variation of unemployment rate among the top 10 countries with the highest number of YouTube channels')
plt.show()


### 13. What is the average urban population percentage in countries with YouTube channels? 

In [ ]:
df_country_analysis['Urban_ratio'] = (df_country_analysis['Urban_population'] / df_country_analysis['Population']) * 100

country_avg_urban = df_country_analysis.groupby('Country')['Urban_ratio'].mean()

overall_avg = country_avg_urban.mean()
print(country_avg_urban)
print("Average urban population percentage in countries with YouTube channels:", overall_avg)

plt.bar(country_avg_urban.index, country_avg_urban.values)
plt.xlabel('Country')
plt.ylabel('Average urban population')
plt.xticks(rotation=90)
plt.show()

### 14. Are there any patterns in the distribution of YouTube channels based on latitude and longitude coordinates? 

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(df['Longitude'], df['Latitude'])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Geographic Distribution of YouTube Channels')
plt.grid(True, alpha=0.3)
plt.show()

x = df['Longitude'].corr(df['Latitude'])
print('Relation between latitudes and longitudes: ',x)

This concludes that there is negligible correlation in distribution of you tube channels based on latitude and longitude.

### 15. What is the correlation between the number of subscribers and the population of a country? 

In [ ]:
correlation_subscribers_population = df['subscribers'].corr(df['Population'])
print(correlation_subscribers_population)

if correlation_subscribers_population > 0.9:
    print("Very high positive correlation")
elif correlation_subscribers_population > 0.7:
    print("High positive correlation")
elif correlation_subscribers_population > 0.5:
    print("Moderate high positive correlation")
elif correlation_subscribers_population > 0.3:
    print("Low positive correlation")
elif correlation_subscribers_population > 0.0:
    print("negligible correlation")
elif correlation_subscribers_population == 0:
    print("No correlation")
elif correlation_subscribers_population > -0.3:
    print("Negligible correlation")
elif correlation_subscribers_population > -0.5:
    print("Low negative correlation")
elif correlation_subscribers_population > -0.7:
    print("Moderate negative correlation")
elif correlation_subscribers_population > -0.9:
    print("High negative correlation")
else:
    print("Very high negative correlation.")

### 16. How do the top 10 countries with the highest number of YouTube channels compare in terms of their total population? 

In [ ]:
plt.bar(top10['Country'], top10['Population'])
plt.xlabel('country')
plt.ylabel('total population')
plt.title("Top 10 countries' total population with highest number of you tube channels")
plt.xticks(rotation = 90)
plt.show()

### 17. Is there a correlation between the number of subscribers gained in the last 30 days and the unemployment rate in a country? 

In [ ]:
result_correlation = df['subscribers_for_last_30_days'].corr(df['Unemployment rate'])
print(result_correlation)

if result_correlation > 0.9:
    print("Very high positive correlation")
elif result_correlation > 0.7:
    print("High positive correlation")
elif result_correlation > 0.5:
    print("Moderate high positive correlation")
elif result_correlation > 0.3:
    print("Low positive correlation")
elif result_correlation > 0.0:
    print("negligible correlation")
elif result_correlation == 0:
    print("No correlation")
elif result_correlation > -0.3:
    print("Negligible correlation")
elif result_correlation > -0.5:
    print("Low negative correlation")
elif result_correlation > -0.7:
    print("Moderate negative correlation")
elif result_correlation > -0.9:
    print("High negative correlation")
else:
    print("Very high negative correlation.")

### 18. How does the distribution of video views for the last 30 days vary across different channel types?

In [ ]:
data_channel_type = df.groupby(['channel_type'])['video_views_for_the_last_30_days'].describe()
print(data_channel_type)

plt.bar(data_channel_type.index, data_channel_type['mean'])
plt.xlabel('Channel type')
plt.ylabel('video views in last 30 days')
plt.xticks(rotation=90)
plt.show()

### 19. Are there any seasonal trends in the number of videos uploaded by YouTube channels?  

In [ ]:
monthly_avg_uploads = df.groupby('created_month')['uploads'].mean()

print(monthly_avg_uploads)

plt.figure(figsize=(10,6))
monthly_avg_uploads.plot(kind='bar')
plt.title('Average Uploads per Channel by Month')
plt.xlabel('Month')
plt.ylabel('Average Number of Uploads')
plt.xticks(rotation=45)
plt.show()

### 20. What is the average number of subscribers gained per month since the creation of YouTube channels till now?

In [ ]:
current_year = 2025
current_month = 6

month_num = {'Jan':1 , 'Feb':2, 'Mar':3, 'Apr' : 4, 'May': 5, "Jun": 6, 'Jul':7 , 'Aug' : 8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
df['months_age'] = (current_year - df['created_year']) * 12 + (current_month - df['created_month'].map(month_num))

df['subscribers_per_month'] = df['subscribers']/df['months_age']

avg_subscribers = df['subscribers_per_month'].mean()
print("Average subscribers gained per month since creation: ",avg_subscribers)